# Encoder-Decoder with Attention

Encoder-Decoderモデルの発展系として、Attentionを追加してみます。  
ついでにEncocderのLSTMもBidirectional LSTMに変えてみましょう。

残念ながら、2017年10月8日時点で、Attentionを一行で追加してくれるような機能はChainerにはありません。  
Attentionの構造とChainerの関数などをきちんと理解し、自分で実装していくことになります。  

ライブラリのインポート、idの系列をembeddingの系列に変える関数は先と一緒です。

In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [3]:
# 可変長単語id系列を可変長単語ベクトル系列へ写像する関数
def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

### Encoder

せっかくAttentionを使うので、EncoderのLSTMをBidirectionalにしてみます。  
Bidirectional LSTMは、すでにChainerに用意されているので、そこを変えるだけです。

In [4]:
class Encoder(Chain):
    def __init__(self, n_layers, n_source_vocab, n_units, dropout):
        super(Encoder, self).__init__()
        with self.init_scope():
            self.source_embed = L.EmbedID(in_size=n_source_vocab, out_size=n_units)
            
            # NStepLSTMをNStepBiLSTMに
            self.encoder_lstm = L.NStepBiLSTM(n_layers=n_layers, in_size=n_units,
                                              out_size=n_units, dropout=dropout)

            self.n_source_vocab = n_source_vocab
            self.n_units = n_units
    
    def __call__(self, source_xs):
        # 単語の系列を単語ベクトルへ
        exs = sequence_embed(self.source_embed, source_xs)
        
        # lstmの初期状態
        hx = None
        cx = None
        
        # lstmで各系列をエンコード
        hy, cy, ys = self.encoder_lstm(hx, cx, exs)
        return hy, cy, ys
        

注意としては、出力のhy, cyのshapeが変わります。  
n_layerのaxisが、forward LSTMとbackward LSTMの分を合わせ、n_layer * 2になります。  
また、ysの各隠れ層の次元数も2倍になります。  
よってDecoderのLSTMの隠れ層の次元数はEmbeddingやEncoderのn_unitsの二倍になります。

### Attention

Attentionを導入します。

今回はAttentionが機械翻訳に初めて適用された、 

・[Bahdanau et al. (2015) Neural Machine Translation by Jointly Learning Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)  

のモデルを書いていきます。

![attention.png](https://raw.githubusercontent.com/kwashio/semi_tutorial/images/attention.png)

画像は[スタンフォード大学の授業のスライド](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture10.pdf)から拝借しました。

Bahdanauのモデルでは、AttentionはLSTMの一番深い層（画像では一番上の層）で展開され、Decoderの隠れ状態を計算する際に使用されます。  
具体的には、$h_t$を計算する際に、

1. 一つ前の隠れ状態$h_{t-1}$からContext vector $c_t$を計算
1. $h_{t-1}$、$c_t$、input vector（画像だと$h_t$の下のベクトル）から$h_t$を計算

という風になります。

次のクラスAttentionでは、$h_{t-1}$から$a_t$を計算し、$c_t$を出力するまでの処理を記述しています。 

式にすると、Encoderのある隠れ状態の$score_i$は、

\begin{equation}
score_i = softmax(W_2 z_i)
\end{equation}
\begin{equation}
z_i = tanh(W_1 (e_i \oplus h_{t-1}))
\end{equation}

ただし、$e_i$はEncoderの時点$i$における隠れ状態です。$\oplus$はベクトルの結合を表しています。

実装は以下の記事などを参考にしました。  
[今更ながらchainerでSeq2Seq（2）〜Attention Model編〜](https://qiita.com/kenchin110100/items/eb70d69d1d65fb451b67)

In [6]:
class Attention(Chain):
    def __init__(self, n_units):
        super(Attention, self).__init__()
        with self.init_scope():
            #eWとdWで上の式のW1を表している。
            # Encoder(BiLSTM)の隠れ状態の線形変換
            self.eW = L.Linear(n_units*2) # Decoderの隠れ層の次元数はEncoderのn_unitsの二倍
            
            # 一つ前のdecoder中間層の線形変換
            self.dW = L.Linear(n_units*2)
            
            
            # スコア計算用の線形変換、上の式のW2
            self.aW = L.Linear(1)
    
    def __call__(self, ehs, dh):
        # 各z_iの計算
        encoder_hidden = self.eW(ehs)
        
        # h_{t-1}の線形変換後のベクトルをbroadcastし、コピーしてencoder_hiddenに足し合わせる。
        decoder_hidden = F.broadcast_to(self.dW(dh), encoder_hidden.shape)
        attention_hidden = F.tanh(encoder_hidden + decoder_hidden)
        
        # scoreの計算。
        scores = F.softmax(self.aW(attention_hidden), axis=0)
        
        # context vectorの計算
        context = F.matmul(F.transpose(scores), ehs)
        # (1 , n_units*2)
        return context

### Decoder(with Attention)

Attentionを考慮したDecoderを書いていきます。  
少し複雑ですが、頑張ってついてきてください。  

Attentionなしの単純なEncoder-Decoderモデルを書いたときは、EncoderもDecoderもNStepLSTMで書くことができました。  
しかし、Attentionを考慮する場合は、Decoderの一番深いレイヤーの各隠れ状態を計算する際に、context vector $c_t$を計算に入れる必要があるため、単純にNStepLSTMを用いることはできません。

つまり、一番深いレイヤーの計算部分は自分で書かなければなりません。  
もう一度、さきほどの図を見てみましょう。

![attention2.png](https://github.com/kwashio/semi_tutorial/blob/images/attention2.png?raw=true)

$c_t$を除く青い隠れ状態の部分はEncoderで計算済みです。  
赤色の隠れ状態はDecoderで計算するのですが、多層のLSTMを考えた時、赤枠の部分はNStepLSTMで計算できます。  
つまり、自分で書かなければいけないのは、Decoderのトップの層の部分ということになります。

Decoderもトップの層もLSTMなのですが、通常のLSTMとは異なり、$h_{t-1}$とinput以外にcontext vector $c_t$を考慮したLSTMです。  
このようなLSTMを記述する際は、レイヤーのLSTMやNStepLSTMではなく、[chainer.functions.lstm](https://docs.chainer.org/en/stable/reference/generated/chainer.functions.lstm.html#chainer.functions.lstm)を使います。

functionのLSTMは、大雑把に入力と出力を書くと、
```
c, h = lstm(previous_c, W1*previous_h + W2*input)
```
という風になっています。つまり、一つ前のcellと隠れ状態、input vectorを渡すと、新しいcellと隠れ状態を返してくれる関数です。  
注意点としては、previous_h、 input vectorはlstm関数に入力される前に、Linearレイヤー（W1, W2）により中間層の4倍の次元のベクトルに変換されなければないことです。  
なぜ4倍なのかというと、これはLSTMの各構成要素である、input gate、forget gate、output gate、new memory cellの計算に対応しています。

このlstm関数はレイヤーのLSTMと異なり、内部にパラメータを持っておらず、計算処理のみを担っています。  
LSTMとしてのパラメータは上の式における、W1とW2に相当します。

ざっくりとlstm関数を理解したところで、ここにcontext vector $c_t$を組み込みます。  
これは、以下のように行います。

```
c, h = lstm(previous_c, W1*previous_h + W2*input + W3*context)
```

これにより、context vectorを考慮しつつ、新たなcellと隠れ状態を計算することができます。  
では、Decoderクラスを書いていきましょう。
後々の処理過程を前に書いたAttentionなしのEncoder-Decoderモデルに合わせるため、出力はNStepLSTMと同じになるようにします。



In [7]:
class Decoder(Chain):
    
    def __init__(self, n_layers, n_target_vocab, n_units, attention, dropout):
        super(Decoder, self).__init__()
        with self.init_scope():
            self.n_target_vocab = n_target_vocab
            self.target_embed = L.EmbedID(n_target_vocab, n_units, dropout)
            self.n_layers = n_layers
            
            # layer数が１かそれ以上かで、NStepLSTMを使うかどうか分岐
            if self.n_layers > 1:
                self.pre_lstm = L.NStepLSTM(self.n_layers -1, n_units, n_units*2, dropout)
            
            # attention
            self.Att = attention
            
            self.dropout = dropout
            
            # topのLSTMの各線形変換（W1, W2, W3）
            # 中間層がn_units*2なので、それを４倍にする。
            self.lstm_input = L.Linear(n_units * 8)
            self.lstm_previous = L.Linear(n_units * 8)
            self.lstm_context = L.Linear(n_units * 8)
            
        
    def __call__(self, hy, cy, ys, target_xs):
        # hy, cy, ysはEncoder(BiLSTM)の出力
        # target_xsは、単語idの系列のリスト
        
        # attention以外の部分の計算
        batch_size = len(ys)
        
        # ターゲット言語の単語idの系列のリストを、embeddingの系列のリストへ
        exs = sequence_embed(self.target_embed, target_xs)
        
        # EncoderのBiLSTMのhy, cyのshapeをDecoderの構造に合わせる。
        # (n_layers*2, batchsize, n_units) -> (n_layers, batchsize, n_units*2)へ
        hy = F.reshape(hy, (self.n_layers, batch_size, -1))
        cy = F.reshape(cy, (self.n_layers, batch_size, -1))
        
        # n_layersが２以上のときは、NStepLSTMにより、一番深い層以外の隠れ状態を計算しておく。
        if self.n_layers > 1:
            unatt_n_layer = self.n_layers - 1
            pre_hy = hy[:unatt_n_layer]
            pre_cy = cy[:unatt_n_layer]
            after_h, after_c, pre_os = self.pre_lstm(pre_hy, pre_cy, exs)
        
        else:
            pre_os = exs
        # pre_osが、一番深い層のLSTMへのinputの系列になる。
        
        # 最終層の計算
        high_hy = hy[self.n_layers - 1]
        high_cy = cy[self.n_layers - 1]
        
        # NStepLSTMと出力を合わせるために、リストを３つ用意
        last_h = [] # 最後の隠れ状態のリスト
        last_c = [] # 最後のcellのリスト
        os = [] # 隠れ状態の系列のリスト
        
        # 各input系列ごとに処理
        for i, pre_eos in enumerate(pre_os):
            h = F.reshape(high_hy[i], (1,-1))
            c = F.reshape(high_hy[i], (1,-1))
            now_ys = ys[i] # 対応するEncoderの隠れ状態の系列 (lenght, n_units*2)
            temp_os = []
            
            # verticalにdropoutがかかるので、dropoutをかける場所はここ
            pre_eos = F.dropout(pre_eos, self.dropout)
            
            
            for x in pre_eos:
                # input vector
                x = F.reshape(x, (1,-1))
                
                # 一つ前の隠れ状態からcontext vectorを計算。
                context = self.Att(now_ys, h)
                
                # 次のセルと隠れ状態を計算する。
                c, h = F.lstm(c,
                              self.lstm_input(x) + self.lstm_previous(h) + self.lstm_context(context))
                
                # 隠れ状態を保存
                temp_os.append(h)
            
            # 最後の隠れ状態、最後のセル、隠れ状態の系列を保存
            last_h.append(h)
            last_c.append(c)
            os.append(F.concat(temp_os, axis=0))
        
        # 出力をNStepLSTMに合わせるために、shapeを変換
        last_h = F.reshape(F.concat(last_h, axis=0), (1, batch_size, -1))
        last_c = F.reshape(F.concat(last_c, axis=0), (1, batch_size, -1))
        
        # n_layerが２以上のときは、NStepLSTMの出力とconcatする。
        if self.n_layers > 1:
            ho = F.concat([after_h, last_h], axis=0)
            co = F.concat([after_c, last_c], axis=0)
        else:
            ho = last_h
            co = last_c
            
        
        return ho, co, os

        

        
                

お疲れ様でした。  
頑張って出力の形を揃えたので、あとの処理はAttentionなしのEncoder-Decoderモデルとほぼ同じです。

### Encoder-Decoder+Attentionモデル

In [17]:
UNK = 0
EOS = 1

In [38]:
class Encoder_Decoder_withAttention(Chain):
    
    def __init__(self, encoder, decoder):
        super(Encoder_Decoder_withAttention, self).__init__()
        with self.init_scope():
            self.encoder = encoder
            self.decoder = decoder
        
            self.W = L.Linear(self.decoder.n_target_vocab)
            
    def __call__(self, xs, target_xs):
        
        eos = self.xp.array([EOS], 'i')
        target_in = [F.concat([eos, y], axis=0) for y in target_xs]
        target_out = [F.concat([y, eos], axis=0) for y in target_xs]
        
        hy, cy, ys = self.encoder(xs)
        _, _, os = self.decoder(hy, cy, ys, target_in)
        
        # loss calculation
        batch_size = len(xs)
        concat_os = F.concat(os, axis=0)
        concat_target_out = F.concat(target_out, axis=0)
        loss = F.sum(F.softmax_cross_entropy(
            self.W(concat_os), concat_target_out, reduce='no')) / batch_size
        
        return loss
    
    def translate(self, xs, max_length = 100):
        batch_size = len(xs)
        
        with chainer.no_backprop_mode(), chainer.using_config('train', False):
            # Encode
            hy, cy, ys = self.encoder(xs)
            
            # decode時のinput用にbatch_size分のEOS=1を用意
            target_xs = self.xp.full((batch_size,1), 1, 'i')
            result = []
            
            ho = hy
            co = cy
            for i in range(max_length):
                # 翻訳（予測）の際は、Encoderの出力をdecoderに入れる(ys)
                ho, co, os = self.decoder(ho, co, ys, target_xs) 
                
                concat_os = F.concat(os, axis=0)
                wy = self.W(concat_os)
                target_xs = self.xp.argmax(wy.data, axis=1).astype('i')
                result.append(target_xs)
                target_xs = F.reshape(target_xs, (-1, 1)).data
            
            result = self.xp.stack(result).T
            # Remove EOS tags
            outs = []
            for y in result:
                inds = np.argwhere(y == EOS)
                if len(inds) > 0:
                    y = y[:inds[0, 0]]
                outs.append(y)
        return outs
            
            
            
            

---

# 実際に動かす

ここでも実際に動かしてみます。  
Attentionなしのモデルを動かしたときは１００文対のみで訓練しましたが、今回は前よりたくさんデータを使って実験してみましょう。  
１５００文対で学習してみます。  
先と同じく、日本語はMecabで分かち書き、英語は小文字化しておきます。

今回も英日翻訳をやっていきます。

今回は訓練データから、適当に３つ文を抜き出し、validation dataとします。  
validation dataで数epochごとに翻訳の品質を試してみます。

### データ読み込み、前処理

In [119]:
# 訓練データ
with open('data_full/ja.txt','r') as f:
    ja = f.read().strip().split('\n')

with open('data_full/en.txt','r') as f:
    en = f.read().strip().split('\n')
    
ja = [s.strip() for s in ja][:1500]
en = [s.strip() for s in en][:1500]

In [120]:
# validationデータ
with open('data_full/val_ja.txt','r') as f:
    val_ja = f.read().strip().split('\n')

with open('data_full/val_en.txt','r') as f:
    val_en = f.read().strip().split('\n')
    
val_ja = [s.strip() for s in val_ja]
val_en = [s.strip() for s in val_en]

In [121]:
# 訓練データで単語id辞書作成
ja_w2id = {'UNK':0, 'EOS':1}
en_w2id = {'UNK':0, 'EOS':1}

ja_vocab = set()
for s in ja:
    ja_vocab.update(s.split(' '))
    
en_vocab = set()
for s in en:
    en_vocab.update(s.split(' '))

for i, w in enumerate(ja_vocab):
    ja_w2id[w] = i+2
    
for i, w in enumerate(en_vocab):
    en_w2id[w] = i+2

In [122]:
# 単語id辞書保存
import pickle
with open('data_full/ja_w2id.dump', 'wb') as f:
    pickle.dump(ja_w2id, f)

with open('data_full/en_w2id.dump', 'wb') as f:
    pickle.dump(en_w2id, f)

In [123]:
# 単語系列をid系列に変換
ja_data = []
for s in ja:
    s = s.split(' ')
    s = [ja_w2id[w] for w in s]
    ja_data.append(np.array(s, 'i'))

en_data = []
for s in en:
    s = s.split(' ')
    s = [en_w2id[w] for w in s]
    en_data.append(np.array(s, 'i'))

In [124]:
# validation dataも
val_ja_data = []
for s in val_ja:
    s = s.split(' ')
    s = [ja_w2id.get(w, 0) for w in s]
    val_ja_data.append(np.array(s, 'i'))
    
val_en_data = []
for s in val_en:
    s = s.split(' ')
    s = [en_w2id.get(w, 0) for w in s]
    val_en_data.append(np.array(s, 'i'))

In [125]:
val_en_data

[array([2601,  275,    0], dtype=int32),
 array([ 885,  377, 2779,    0,    0], dtype=int32),
 array([1846, 2438, 1888,    0, 1279, 1888, 1043,    0], dtype=int32)]

今回は英語のvalidation dataに未知語（UNK＝０）があります。

In [126]:
for s in zip(val_en, val_ja):
    print(s)

('i give advice.', '私 が アドバイス を する')
('he listened to today’s news.', '彼 は 今日 の ニュース を 聞い た')
('we use the trees in the japanese mountains.', 'われわれ は 日本 の 山 の 木 を 使い ます')


## モデル用意

今回は以下のようなモデルを学習してみましょう。

- レイヤー数は2.
- n_units(embeddingの次元数)は50.
- dropout rateは0.2
- 最適化はAdam

In [127]:
n_layers = 2
n_units = 50
dropout = 0.2

In [128]:
# 英語の文のEncoder
encoder = Encoder(n_layers=n_layers, n_source_vocab=len(en_w2id),
                  n_units=n_units, dropout=dropout)

# Attention
attention = Attention(n_units=n_units)

# 日本語の文のDecoder
decoder = Decoder(n_layers=n_layers, n_target_vocab=len(ja_w2id),
                  n_units=n_units, attention=attention, dropout=dropout)

# Encoder-Decoderモデル。
model = Encoder_Decoder_withAttention(encoder, decoder)

In [129]:
optimizer = optimizers.Adam(0.01)
optimizer.setup(model)

## 学習

また、20epochほど回してみて、5epochごとにどれくらいうまくフィッティングできているかを確かめましょう。  
今回はミニバッチサイズは１００としましょう。

In [130]:
# idから単語への辞書。
ja_id2w = {i:w for w, i in ja_w2id.items()}
en_id2w = {i:w for w, i in en_w2id.items()}

# 中間報告用の関数
def translate_print(en_array, ja_array, en_id2w, ja_id2w):
    print('=====中間報告=====')
    for en, ja in zip(en_array, ja_array):
        en_s = [en_id2w[i] for i in en]
        print(' '.join(en_s))
        
        ja_s = [ja_id2w[i] for i in ja]
        print('>>> '+' '.join(ja_s))
    print('\n')

In [131]:
# 訓練データ総数
n_train = len(ja_data)

# ミニバッチサイズ
batchsize = 100

# エポック数
n_epoch =20

# ミニバッチ化のために、array化しておく。dtypeはobject
ja_data = np.array(ja_data)
en_data = np.array(en_data)

for epoch in range(n_epoch): # epochのループ
    sum_loss = 0
    # ミニバッチの用意。
    perm = np.random.permutation(n_train)
    for i in range(0, n_train, batchsize):
        ja_batch = ja_data[perm[i:i+batchsize]]
        en_batch = en_data[perm[i:i+batchsize]]
        
        # lossの計算
        loss = model(en_batch, ja_batch)
        sum_loss += loss.data
        # backpropatgation
        optimizer.target.cleargrads() # 勾配のリセット
        loss.backward() # 逆伝搬する誤差を計算
        optimizer.update() # パラメータを更新
    
    log = '# epoch: {}, loss: {}'.format(epoch+1, sum_loss)
    print(log)
    
    # 5epochごとに翻訳を出力。
    if (epoch+1)%5 == 0:
        result = model.translate(val_en_data)
        translate_print(val_en_data, result, en_id2w, ja_id2w)
        
    
    

# epoch: 1, loss: 947.2382125854492
# epoch: 2, loss: 813.2485122680664
# epoch: 3, loss: 782.7784729003906
# epoch: 4, loss: 763.1520195007324
# epoch: 5, loss: 746.2089996337891
=====中間報告=====
i give UNK
>>> 彼 が の の
he listened to UNK UNK
>>> 彼 が の の を
we use the UNK in the japanese UNK
>>> 彼 が の の の を


# epoch: 6, loss: 729.8228607177734
# epoch: 7, loss: 712.6900482177734
# epoch: 8, loss: 692.3330307006836
# epoch: 9, loss: 655.3534851074219
# epoch: 10, loss: 611.0483818054199
=====中間報告=====
i give UNK
>>> 私 が 人 に い ます
he listened to UNK UNK
>>> 彼 が 、 、 人 を ます
we use the UNK in the japanese UNK
>>> 私 は 、 、 人 に い ます


# epoch: 11, loss: 580.491626739502
# epoch: 12, loss: 554.8106422424316
# epoch: 13, loss: 534.7021408081055
# epoch: 14, loss: 513.9462642669678
# epoch: 15, loss: 492.64785957336426
=====中間報告=====
i give UNK
>>> 私 が 私 に い ます
he listened to UNK UNK
>>> 彼 が X を し ます
we use the UNK in the japanese UNK
>>> 私 は 、 、 １ ０ ０ ０ 人 に は い ます


# epoch: 16, loss: 473.075893402

今回はここで打ち切ります。  
lossは下がっていってますが、フィッティングはまだまだという感じです。  
ボキャブラリが増えたからかもしれません。  
一応、最初は非文が生成されていますが、だんだん日本語の文らしきものが生成されるようにはなっていってます。  

---

# まとめ

お疲れ様でした。  
Attentionを実装してみました。  

今回のように、フレームワークに予め用意されていないコンポーネントを実装するのは大変ですが、研究では確実に必要になってきます。  
そのようなときは次のプロセスを踏むと良いでしょう。

1. 実装したいモデルの構造を、論文を読み込んだりしてちゃんと理解する。
2. 誰かが実装している場合はそれをパクる。
3. 無い場合は自分で書く。

２において、自分がメインで使っているフレームワークとは別のフレームワークで実装されている、ということもあると思います。  
そのようなときに、色んなフレームワークについて最低限コードは読めるぐらいになっておくと良いでしょう。  
自分で書くのはともかく、読めるようになるコストは低いと思います。

最近ではChainerと似た思想で作られたフレームワークであるPyTorchによる実装などが増えているようです。  
書き方はChainerとほぼ同じですし、チュートリアルもとても充実しているので、まずこのあたりに触れてみるのはいかがでしょうか。  
http://pytorch.org/docs/master/  
http://pytorch.org/tutorials/